In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import time
import glob
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics 

#matplot lib notebooks display command
%matplotlib inline

#SET OPTIONS
pd.options.display.float_format = '{:,.2f}'.format
#GLOBAL VARIABLES
today = datetime.date.today()
#cwd = os.getcwd()

In [2]:
#load dataset
non_auto_df = pd.read_csv("../Data/non_automated_invoices_sample.csv", encoding='utf-8')
non_auto_df.dtypes

Unnamed: 0                 int64
ID                        object
COURTESYSENT__C             bool
COLLECTOR                 object
OUTSOURCE_PARTNER         object
RESELLER                  object
COLLECTION_STATUS         object
HAS_TAX_EXEMP               bool
HAS_REPLACEMENT_ORDER       bool
ESCALATION_STATUS         object
COLLECTOR_FORECAST        object
BILLING_COUNTRY           object
BILLING_LANGUAGE          object
PAYMENT_TERM              object
PAYMENT_TYPE              object
SFBILL__INVOICEAGE__C      int64
HAS_30DAY_EXTXN             bool
PAST_DUE_FLG                bool
LATE_PMT                    bool
DAYS_TO_PAY                int64
CURRENCY_CODE             object
TOTAL_TAX                float64
TOTAL_AMOUNT             float64
dtype: object

## Definition of features



'ID'- identification [DID NOT USE]

'COURTESYSENT__C' -         ???

'COLLECTOR' - is this name of person collecting ?

'OUTSOURCE-PARTNER' - name of outsource partner ?

'RESELLER' - name of reseller?

'COLLECTION_STATUS' - status of collection (yes or no). Is this updated throughout process? [DID NOT USE]

'HAS-TAX-EXEMPT' - has tax exemption

'HAS-REPLACEMENT-ORDER' - has an order already for next billing cycle

'ESCALATION_STATUS' -I assume this is updated throughout process [DID NOT USE]

'COLLECTOR_FORECAST' -I assume this is updated throughout process [DID NOT USE]

'BILLING_COUNTRY' - country of origin for customer

'BILLING_LANGUAGE' -language of billing for customer

'PAYMENT_TERM' - is this a number of days? 

'PAYMENT_TYPE' - how customer is paying bill, you've already eliminated credit cards right?

'SFBILL___INVOICEAGE__C' INVOICE AGE [REMOVE]

'HAS_30DAY_EXTXN' - when is this information determined?  
'PAST_DUE_FLAG' - when is this updated? [DID NOT USE]

'LATE_PMT' - if customer made late payment [DID NOT USE]

'DAYS_TO_PAY' - days left before due?  [DID NOT USE]

'CURRENCY-CODE' - is this what kind of currency they are using?

'TOTAL-TAX' - total tax on product customer bought

'TOTAL AMOUNT' - total customer spent on product



In [3]:
#Normalize Numeric fields

def pre_process(df, dropcols=None):
    '''
    The pre_process function that will return a transformed dataframe
    Notes:
        1) dropcols should be a list
        2) integer varibles are transformed using mean normalization
    '''
    #drop columns input by the user
    if dropcols is not None:
        df = df.drop(columns=dropcols)
    print("columns dropped")
    
    #loop through remaining columns and apply the appropreate transform
    for i in df.columns:
        print(i)
        '''
        #mean normalization for int variables
        if df[i].dtype == "int64" or df[i].dtype == 'float64':
            mean = df[i].mean()
            if (df[i].max() - df[i].min()) == 0:
                denom = 1
                print("useless int column", i)
                break
            else:
                denom = (df[i].max() - df[i].min())
            df[i] = df[i].apply(lambda x: float(x - mean) / denom)
        '''
        #convert bool values to 1 and 0
        if df[i].dtype == "bool":
            df[i] = df[i].map({True: 1, False: 0})
        elif df[i].dtype == "object":
            df = pd.get_dummies(df, columns=[i], prefix = [i], dtype="int8") #, sparse=True
        
    return df



In [4]:
print("executed")
clean_df = pre_process(non_auto_df, dropcols=['ID', 'COLLECTION_STATUS', \
                                              'ESCALATION_STATUS', 
                                             'PAST_DUE_FLG', 'COLLECTOR_FORECAST'])
#clean_df.to_csv('../Data/clean_df.csv')
#dtypes_df = clean_df.dtypes.to_frame('dtypes').reset_index()
#dtypes_df.to_csv('../Data/clean_dtypes.csv')
print("done")

executed
columns dropped
Unnamed: 0
COURTESYSENT__C
COLLECTOR
OUTSOURCE_PARTNER
RESELLER
HAS_TAX_EXEMP
HAS_REPLACEMENT_ORDER
BILLING_COUNTRY
BILLING_LANGUAGE
PAYMENT_TERM
PAYMENT_TYPE
SFBILL__INVOICEAGE__C
HAS_30DAY_EXTXN
LATE_PMT
DAYS_TO_PAY
CURRENCY_CODE
TOTAL_TAX
TOTAL_AMOUNT
done


In [6]:
X = clean_df.loc[:, clean_df.columns != "DAYS_TO_PAY"]
y = clean_df['DAYS_TO_PAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
print("done")

done


In [ ]:
import sklearn.svm as svm
import sklearn.metrics as metrics
print("executed")
#below code from sklearn documentaion
# #############################################################################
# Fit regression model
svr_rbf = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = svm.SVR(kernel='linear', C=100, gamma='auto')
svr_poly = svm.SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
#end documentation code

print("svrs created")
clf_rbf = svr_rbf.fit(X_train, y_train)
clf_lin = svr_lin.fit(X_train, y_train)
clf_poly = svr_poly.fit(X_train, y_train)
print("classifiers created")
pred_rbf = svr_rbf.predict(X_valid)
pred_lin = svr_lin.predict(X_valid)
pred_poly = svr_poly.predict(X_valid)


print("Mean absolute error with rbf kernel is ", metrics.mean_absolute_error(y_valid, pred_rbf))
print("Mean absolute error with linear kernel is ", metrics.mean_absolute_error(y_valid, pred_lin))
print("Mean absolute error with polynomial kernel is ", metrics.mean_absolute_error(y_valid, pred_poly))

executed
svrs created
